In [148]:
#form to save in the database
Radiologydb=pd.DataFrame({'Filepath': DICOMlist, 'ImageID':list(map(lambda x : SOPInstanceUIDFunction(x), range(len(DICOMlist)))), 
                                  'SeriesID':list(map(lambda x : SeriesInstanceFunction(x), range(len(DICOMlist)))), 
                                  'RadiologyOccurrenceID':list(map(lambda x : StudyInstanceUIDFunction(x), range(len(DICOMlist)))), 
                                  'PatientID':list(map(lambda x : PatientIDFunction(x), range(len(DICOMlist)))), 
                                  'ImageResolutionRows':list(map(lambda x : RowsFunction(x), range(len(DICOMlist)))), 
                                  'ImageResolutionColumns':list(map(lambda x : ColumnsFunction(x), range(len(DICOMlist)))), 
                                  'SliceThickness':list(map(lambda x : SliceThicknessFunction(x), range(len(DICOMlist)))),
                                 'ProtocolName':list(map(lambda x : ProtocolName(x), range(len(DICOMlist)))),
                                 'PerformedProcedureStepDescription':list(map(lambda x : PerformedProcedureStepDescription(x), range(len(DICOMlist)))),
                                 'BodyPartExamined':list(map(lambda x : BodyPartExamined(x), range(len(DICOMlist)))),
                                 'Modality':list(map(lambda x : Modality(x), range(len(DICOMlist)))),
                                 'SeriesDescription':list(map(lambda x : SeriesDescription(x), range(len(DICOMlist)))),
                                 'StudyDescription':list(map(lambda x : StudyDescription(x), range(len(DICOMlist)))),
                                 'Manufacturer':list(map(lambda x : Manufacturer(x), range(len(DICOMlist))))})
grouped=Radiologydb[['ImageID','SeriesID']].groupby('SeriesID')
PhaseTotalNo=grouped.count()
PhaseTotalNo.columns = ['PhaseTotalNo']
Radiologydb['ImageNo']=Radiologydb.groupby(['SeriesID']).cumcount()+1
Radiologydb=pd.merge(Radiologydb, PhaseTotalNo, on='SeriesID')

In [1]:
import os
import pydicom
import pandas as pd
import numpy as np
from datetime import datetime
import pyodbc

In [2]:
# Identify DICOM files under 'DICOMpath' directory
DICOMpath='C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2'
DICOMlist = []
for (path, dir, files) in os.walk(DICOMpath):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.dcm':
            filePath = "{0}/{1}".format(path, filename)
            DICOMlist.append(filePath)
Metadata=list(map(lambda x : pydicom.filereader.dcmread(x), DICOMlist))

C:\Users\ABMI\Anaconda3\lib\site-packages\pydicom\charset.py:658: UserWarning: Incorrect value for Specific Character Set 'ISO IR 6' - assuming 'ISO_IR 6'
  _warn_about_invalid_encoding(encoding, patched)


In [3]:
def ColumnsFunction(x):
    try:
        return Metadata[x].Columns
    except:
        return 'NA'

In [4]:
def RowsFunction(x):
    try:
        return Metadata[x].Rows
    except:
        return 'NA'

In [5]:
def SeriesInstanceFunction(x):
    try:
        return Metadata[x].SeriesInstanceUID
    except:
        return 'NA'

In [6]:
def SOPInstanceUIDFunction(x):
    try:
        return Metadata[x].SOPInstanceUID
    except:
        return 'NA'

In [7]:
def StudyInstanceUIDFunction(x):
    try:
        return Metadata[x].StudyInstanceUID
    except:
        return 'NA'

In [8]:
def PatientIDFunction(x):
    try:
        return Metadata[x].PatientID
    except:
        return 'NA'

In [9]:
def SliceThicknessFunction(x):
    try:
        return Metadata[x].SliceThickness
    except:
        return 'NA'

In [10]:
def SeriesDescription(x):
    try:
        return Metadata[x].SeriesDescription
    except:
        return 'NA'

In [11]:
#Create RadiologyImageTable without columns of ImageOrientation, RadiologyPhaseConceptID
RadiologyImageTable=pd.DataFrame({'Filepath': DICOMlist, 'ImageID':list(map(lambda x : SOPInstanceUIDFunction(x), range(len(DICOMlist)))), 
                                  'SeriesID':list(map(lambda x : SeriesInstanceFunction(x), range(len(DICOMlist)))), 
                                  'RadiologyOccurrenceID':list(map(lambda x : StudyInstanceUIDFunction(x), range(len(DICOMlist)))), 
                                  'PatientID':list(map(lambda x : PatientIDFunction(x), range(len(DICOMlist)))), 
                                  'ImageResolutionRows':list(map(lambda x : RowsFunction(x), range(len(DICOMlist)))), 
                                  'ImageResolutionColumns':list(map(lambda x : ColumnsFunction(x), range(len(DICOMlist)))), 
                                  'SliceThickness':list(map(lambda x : SliceThicknessFunction(x), range(len(DICOMlist))))})
RadiologyImageTable["Phase_Concept_ID"] = ""
RadiologyImageTable["Image_Orientation_Concept_ID"] = ""
grouped=RadiologyImageTable[['ImageID','SeriesID']].groupby('SeriesID')
PhaseTotalNo=grouped.count()
PhaseTotalNo.columns = ['PhaseTotalNo']
RadiologyImageTable['ImageNo']=RadiologyImageTable.groupby(['SeriesID']).cumcount()+1
RadiologyImageTable=pd.merge(RadiologyImageTable, PhaseTotalNo, on='SeriesID')

In [12]:
RadiologyImageTable

,Filepath,ImageID,SeriesID,RadiologyOccurrenceID,PatientID,ImageResolutionRows,ImageResolutionColumns,SliceThickness,Phase_Concept_ID,Image_Orientation_Concept_ID,ImageNo,PhaseTotalNo
0,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\2008...,9.1.48564849485154484851.1.1182212.3,9.1.48564849485154484851.1,9.1.48564849485154484851,0710537,426,568,NA,,,1,2
1,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\2008...,9.1.48564849485154484851.1.1182220.4,9.1.48564849485154484851.1,9.1.48564849485154484851,0710537,426,568,NA,,,2,2
2,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\2008...,9.1.48564849485254495754.1.1183323.2,9.1.48564849485254495754.1,9.1.48564849485254495754,0931596,426,568,NA,,,1,2
3,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\2008...,9.1.48564849485254495754.1.1183328.3,9.1.48564849485254495754.1,9.1.48564849485254495754,0931596,426,568,NA,,,2,2
4,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\2008...,1.3.12.2.1107.5.1.4.50373.30000008010421505873...,1.3.12.2.1107.5.1.4.50373.30000008010421505873...,1.2.410.2000010.82.220.10801070004,0956645,512,512,4.5,,,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...
2575,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\LIDC...,1.3.6.1.4.1.14519.5.2.1.6279.6001.245155194022...,1.3.6.1.4.1.14519.5.2.1.6279.6001.154696925868...,1.3.6.1.4.1.14519.5.2.1.6279.6001.176996323847...,LIDC-IDRI-0029,2758,2465,NA,,,1,1
2576,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\LIDC...,1.3.6.1.4.1.14519.5.2.1.6279.6001.976782320365...,1.3.6.1.4.1.14519.5.2.1.6279.6001.115231804715...,1.3.6.1.4.1.14519.5.2.1.6279.6001.256178497642...,LIDC-IDRI-0030,2022,1736,NA,,,1,4
2577,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\LIDC...,1.3.6.1.4.1.14519.5.2.1.6279.6001.185557258541...,1.3.6.1.4.1.14519.5.2.1.6279.6001.115231804715...,1.3.6.1.4.1.14519.5.2.1.6279.6001.256178497642...,LIDC-IDRI-0030,2022,1736,NA,,,2,4
2578,C:/Users/ABMI/Desktop/R-CDM/R-CDM sample2\LIDC...,1.3.6.1.4.1.14519.5.2.1.6279.6001.640511412671...,1.3.6.1.4.1.14519.5.2.1.6279.6001.115231804715...,1.3.6.1.4.1.14519.5.2.1.6279.6001.256178497642...,LIDC-IDRI-0030,2022,1736,NA,,,3,4


In [13]:
def StudyDateFunction(x):
    try:
        return Metadata[x].StudyDate
    except:
        return ''
def StudyTimeFunction(x):
    try:
        return Metadata[x].StudyTime[0:6]
    except:
        return ''
    
StudyDate=list(map(lambda x : StudyDateFunction(x), range(len(DICOMlist))))
StudyTime=list(map(lambda x : StudyTimeFunction(x), range(len(DICOMlist))))

def StudyDateTime(x):
    DateTime=StudyDate[x]+StudyTime[x]
    try:
        return str(datetime.strptime(DateTime, '%Y%m%d%H%M%S'))
    except:
        try:
            return str(datetime.strptime(DateTime, '%Y%m%d'))
        except:
            return str(datetime.strptime(DateTime, ''))


In [14]:
def BodyPartExamined(x):
    try:
        return Metadata[x].BodyPartExamined
    except:
        return 'NA'

In [15]:
def ProtocolName(x):
    try:
        return Metadata[x].ProtocolName
    except:
        return 'NA'

In [16]:
def StudyDescription(x):
    try:
        return Metadata[x].StudyDescription
    except:
        return 'NA'

In [17]:
def PerformedProcedureStepDescription(x):
    try:
        return Metadata[x].PerformedProcedureStepDescription
    except:
        return 'NA'

In [18]:
def Manufacturer(x):
    try:
        return Metadata[x].Manufacturer
    except:
        return 'NA'

In [19]:
def Modality(x):
    try:
        return Metadata[x].Modality
    except:
        return 'NA'

In [20]:
RadiologyOccurrenceTable=pd.DataFrame({'RadiologyOccurrenceID':list(map(lambda x : StudyInstanceUIDFunction(x), range(len(DICOMlist)))),
                                       'StudyDateTime':list(map(lambda x : StudyDateTime(x), range(len(DICOMlist)))), 
                                       'PatientID':list(map(lambda x : PatientIDFunction(x), range(len(DICOMlist)))), 
                                       'Manufacturer':list(map(lambda x : Manufacturer(x), range(len(DICOMlist)))), 
                                       'Modality':list(map(lambda x : Modality(x), range(len(DICOMlist)))),
                                       'BodyPartExamined':list(map(lambda x : BodyPartExamined(x), range(len(DICOMlist)))),
                                       'ProtocolName':list(map(lambda x : ProtocolName(x), range(len(DICOMlist)))),
                                       'StudyDescription':list(map(lambda x : StudyDescription(x), range(len(DICOMlist)))),
                                       'PerformedProcedureStepDescription':list(map(lambda x : PerformedProcedureStepDescription(x), range(len(DICOMlist))))
                                      })
RadiologyOccurrenceTable['Radiology_Comment']=''

In [21]:
imageTotalNo=RadiologyOccurrenceTable[['RadiologyOccurrenceID', 'PatientID']].groupby('RadiologyOccurrenceID').count()
imageTotalNo.columns = ['imageTotalNo']
RadiologyOccurrenceTable=RadiologyOccurrenceTable.groupby('RadiologyOccurrenceID').first().reset_index()
RadiologyOccurrenceTable=pd.merge(RadiologyOccurrenceTable, imageTotalNo, on='RadiologyOccurrenceID')

In [22]:
RadiologyOccurrenceTable

,RadiologyOccurrenceID,StudyDateTime,PatientID,Manufacturer,Modality,BodyPartExamined,ProtocolName,StudyDescription,PerformedProcedureStepDescription,Radiology_Comment,Radiology_Protocol_Concept_ID,imageTotalNo
0,1.2.392.200036.9116.2.6.1.48.1214868379.129460...,2011-01-10 05:48:22,AN_ID_20160905152254,TOSHIBA,CT,NA,Brain(1sec) 5mm (0.5mm x 32),HEAD,NA,,,33
1,1.2.392.200036.9116.2.6.1.48.1214868379.129460...,2011-01-10 06:20:33,AN_ID_20160905152340,TOSHIBA,CT,NA,NECK ANGIO 3mm(0.5mm x 64),NECK,NA,,,263
2,1.2.392.200036.9116.2.6.1.48.1214868379.129510...,2011-01-16 00:05:54,AN_ID_20160905154229,TOSHIBA,CT,NA,Brain HCT 5mm (0.5mm x 32),HEAD,NA,,,54
3,1.2.392.200036.9116.2.6.1.48.1214868379.133568...,2012-04-29 15:57:11,AN_ID_20160921122518,TOSHIBA,CT,NA,Brain(1sec) 5mm (0.5mm x 32),HEAD,NA,,,29
4,1.2.392.200036.9116.2.6.1.48.1214868379.133568...,2012-04-29 16:00:15,AN_ID_20160921122438,TOSHIBA,CT,NA,NECK ANGIO 3mm(0.5mm x 64),NECK,NA,,,249
...,...,...,...,...,...,...,...,...,...,...,...,...
56,1.3.6.1.4.1.14519.5.2.1.6279.6001.675114969651...,2000-01-01 00:00:00,LIDC-IDRI-0023,GE MEDICAL SYSTEMS,DX,CHEST,NA,NA,NA,,,2
57,1.3.6.1.4.1.14519.5.2.1.6279.6001.774731473056...,2000-01-01 00:00:00,LIDC-IDRI-0020,GE MEDICAL SYSTEMS,DX,CHEST,NA,NA,NA,,,2
58,1.3.6.1.4.1.14519.5.2.1.6279.6001.960701544671...,2000-01-01 00:00:00,LIDC-IDRI-0010,Philips Medical Systems,CR,CHEST,NA,NA,NA,,,1
59,9.1.48564849485154484851,2008-01-07 18:22:12,0710537,DATA HUB,AP,,,,NA,,,2


In [23]:
import pandas as pd
answer=pd.read_csv('C:/Users/ABMI/Desktop/Radiology 매핑/Loinc_2.66_LoincRsnaRadiologyPlaybook_2.66/answer.csv')

#contrast, PET+CT, SPECT, Functional 있는 건 빼고 매핑할거야
i=answer[answer['LongCommonName'].str.contains("contrast")].index
answer=answer.drop(i)

j=answer[answer['LongCommonName'].str.contains("PET")].index
answer=answer.drop(j)

k=answer[answer['LongCommonName'].str.contains("SPECT")].index
answer=answer.drop(k)

l=answer[answer['LongCommonName'].str.contains("Functional")].index
answer=answer.drop(l)

In [24]:
data=pd.DataFrame({'BodyPartExamined':list(map(lambda x : BodyPartExamined(x), range(len(DICOMlist)))),
                   'ProtocolName':list(map(lambda x : ProtocolName(x), range(len(DICOMlist)))), 
                   'StudyDescription':list(map(lambda x : StudyDescription(x), range(len(DICOMlist)))), 
                   'PerformedProcedureStepDescription':list(map(lambda x : PerformedProcedureStepDescription(x), range(len(DICOMlist)))), 
                   'Modality':list(map(lambda x : Modality(x), range(len(DICOMlist))))
                  })
data=data.drop_duplicates()

#clean data modality (BM->BMD)
data=data.replace({'Modality': 'BM'}, {'Modality': 'BMD'})

data['information']=data['ProtocolName']+","+data['PerformedProcedureStepDescription']+","+data['BodyPartExamined']+","+data['StudyDescription']+","+data['Modality']

In [25]:
#clean answer (move modality from LongCommonName) (replace Views from LongCommonName)
def SepModality(x):
    return x.split(' ', 1)[0]
answer['Modality']=answer['LongCommonName'].apply(SepModality)
answer['LongCommonName']=answer['LongCommonName'].str.replace(' Views', '')

#clean answer dataframe
answer['LongCommonName']=answer['LongCommonName'].str.replace('CT ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('DXA ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('MG ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('MR ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('NM ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('PT ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('RF ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('US ', '')
answer['LongCommonName']=answer['LongCommonName'].str.replace('XR ', '')

In [26]:
#clean data newModality
def modality(x):
    if x=='CR' or x=='DX':
        return 'XR'
    elif x=='BM' or x=='DXA':
        return 'DXA'
    elif x=='CT' or x=='MG' or x=='MR' or x=='NM' or x=='PT' or x=='RF' or x=='US':
        return x
    else:
        return 'OT'
data['newModality']=data['Modality'].apply(modality)

In [27]:
#lowercase answerLongCommonName, datainformation
def lower(x):
    return x.lower()
data['information']=data['information'].apply(lower)
answer['LongCommonName']=answer['LongCommonName'].apply(lower)

In [28]:
#modify data information (c-spine : cervical spine) (remove joint, bone)
data['information']=data['information'].str.replace('c-spine', 'cervicalspine')
answer['LongCommonName']=answer['LongCommonName'].str.replace('cervical spine', 'cervicalspine')
data['information']=data['information'].str.replace('l-spine', 'lumbarspine')
data['information']=data['information'].str.replace('l - spine', 'lumbarspine')

answer['LongCommonName']=answer['LongCommonName'].str.replace('lumbar spine', 'lumbarspine')
data['information']=data['information'].str.replace('joint', '')
data['information']=data['information'].str.replace('bone', '')
data['information']=data['information'].str.replace('-', ' ')
data['information']=data['information'].str.replace('+', ' ')
data['information']=data['information'].str.replace('^', ' ')

In [29]:
#divide answer into modality group
CT=list(answer.loc[answer['Modality']=='CT']['LongCommonName'])
DXA=list(answer.loc[answer['Modality']=='DXA']['LongCommonName'])
MG=list(answer.loc[answer['Modality']=='MG']['LongCommonName'])
MR=list(answer.loc[answer['Modality']=='MR']['LongCommonName'])
NM=list(answer.loc[answer['Modality']=='NM']['LongCommonName'])
PT=list(answer.loc[answer['Modality']=='PT']['LongCommonName'])
RF=list(answer.loc[answer['Modality']=='RF']['LongCommonName'])
US=list(answer.loc[answer['Modality']=='US']['LongCommonName'])
XR=list(answer.loc[answer['Modality']=='XR']['LongCommonName'])

In [30]:
import nltk
def map(row):
    if row['newModality']=='MR':
        a=list(set([x if x!='head' else 'brain' for x in nltk.word_tokenize(row['information'])])&set(MR))
        if len(a)==1:
            return(str(int(answer.loc[(answer['LongCommonName']==''.join(a))&(answer['Modality']=='MR')]['radiologyProtocolConceptId'])))
        elif len(a)==2:
            return(str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='MR')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[1])&(answer['Modality']=='MR')]['radiologyProtocolConceptId'])))
        elif len(a)==3:
            return(str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='MR')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[1])&(answer['Modality']=='MR')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='MR')]['radiologyProtocolConceptId'])))
        elif len(a)>=4:
            return(str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='MR')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[1])&(answer['Modality']=='MR')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='MR')]['radiologyProtocolConceptId'])))
        else:
            return ''
    elif row['newModality']=='CT':
        a=list(set([x if x!='brain' else 'head' for x in nltk.word_tokenize(row['information'])])&set(CT))
        if len(a)==1:
            return(str(int(answer.loc[(answer['LongCommonName']==''.join(a))&(answer['Modality']=='CT')]['radiologyProtocolConceptId'])))
        elif len(a)==2:
            return(str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='CT')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[1])&(answer['Modality']=='CT')]['radiologyProtocolConceptId'])))
        elif len(a)==3:
            return(str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='CT')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[1])&(answer['Modality']=='CT')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='CT')]['radiologyProtocolConceptId'])))
        elif len(a)>=4:
            return(str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='CT')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[1])&(answer['Modality']=='CT')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='CT')]['radiologyProtocolConceptId'])))
        else:
            return ''
    elif row['newModality']=='DXA':
        a=list(set(nltk.word_tokenize(row['information']))&set(DXA))
        if len(a)==1:
            return(str(int(answer.loc[(answer['LongCommonName']==''.join(a))&(answer['Modality']=='DXA')]['radiologyProtocolConceptId'])))
        elif len(a)==2:
            return(str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='DXA')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[1])&(answer['Modality']=='DXA')]['radiologyProtocolConceptId'])))
        else:
            return ''
    elif row['newModality']=='MG':
        a=list(set(nltk.word_tokenize(row['information']))&set(MG))
        if len(a)==1:
            return(str(int(answer.loc[(answer['LongCommonName']==''.join(a))&(answer['Modality']=='MG')]['radiologyProtocolConceptId'])))
        elif len(a)==2:
            return(str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='MG')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[1])&(answer['Modality']=='MG')]['radiologyProtocolConceptId'])))
        else:
            return ''
    elif row['newModality']=='NM':
        a=list(set(nltk.word_tokenize(row['information']))&set(NM))
        if len(a)==1:
            return(str(int(answer.loc[(answer['LongCommonName']==''.join(a))&(answer['Modality']=='NM')]['radiologyProtocolConceptId'])))
        elif len(a)==2:
            return(str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='NM')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[1])&(answer['Modality']=='NM')]['radiologyProtocolConceptId'])))
        else:
            return ''
    elif row['newModality']=='PT':
        a=list(set(nltk.word_tokenize(row['information']))&set(PT))
        if len(a)==1:
            return(str(int(answer.loc[(answer['LongCommonName']==''.join(a))&(answer['Modality']=='PT')]['radiologyProtocolConceptId'])))
        elif len(a)==2:
            return(str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='PT')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[1])&(answer['Modality']=='PT')]['radiologyProtocolConceptId'])))
        else:
            return ''
    elif row['newModality']=='RF':
        a=list(set(nltk.word_tokenize(row['information']))&set(RF))
        if len(a)==1:
            return(str(int(answer.loc[(answer['LongCommonName']==''.join(a))&(answer['Modality']=='RF')]['radiologyProtocolConceptId'])))
        elif len(a)==2:
            return(str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='RF')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[1])&(answer['Modality']=='RF')]['radiologyProtocolConceptId'])))
        else:
            return ''
    elif row['newModality']=='US':
        a=list(set(nltk.word_tokenize(row['information']))&set(US))
        if len(a)==1:
            return(str(int(answer.loc[(answer['LongCommonName']==''.join(a))&(answer['Modality']=='US')]['radiologyProtocolConceptId'])))
        elif len(a)==2:
            return(str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='US')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[1])&(answer['Modality']=='US')]['radiologyProtocolConceptId'])))
        else:
            return ''
    elif row['newModality']=='XR':
        a=list(set(nltk.word_tokenize(row['information']))&set(XR))
        if len(a)==1:
            return(str(int(answer.loc[(answer['LongCommonName']==''.join(a))&(answer['Modality']=='XR')]['radiologyProtocolConceptId'])))
        elif len(a)==2:
            return(str(int(answer.loc[(answer['LongCommonName']==a[0])&(answer['Modality']=='XR')]['radiologyProtocolConceptId']))+', '+str(int(answer.loc[(answer['LongCommonName']==a[1])&(answer['Modality']=='XR')]['radiologyProtocolConceptId'])))
        else:
            return ''
    else:
        return ''
 

In [31]:
data['Protocol_Concept_ID']=data.apply(map, axis=1)

In [32]:
data

,BodyPartExamined,ProtocolName,StudyDescription,PerformedProcedureStepDescription,Modality,information,newModality,Protocol_Concept_ID
0,,,,NA,AP,",na,,,ap",OT,
4,HEAD,Brain_without,Head^Brain_without (Adult),C-T Brain Routin,CT,"brain_without,c t brain routin,head,head brai...",CT,3004287
17,UP_GI,NA,NA,NA,RF,"na,na,up_gi,na,rf",RF,
23,ABDOMEN,NA,NA,RP5101,CR,"na,rp5101,abdomen,na,cr",XR,3053099
25,CHEST AP,NA,NA,RP3110,CR,"na,rp3110,chest ap,na,cr",XR,3010713
36,Chest,NA,RP3110,NA,CR,"na,na,chest,rp3110,cr",XR,3010713
37,KUB,NA,NA,RP5200,CR,"na,rp5200,kub,na,cr",XR,
38,HEAD,00_NEURO_BRAIN_angio,Head^00_NEURO_BRAIN_angio (Adult),C-T Brain Angio c Contrast,CT,"00_neuro_brain_angio,c t brain angio c contra...",CT,3004287
115,HEAD,00_NEURO_BRAIN_angio,Head^00_NEURO_BRAIN_angio (Adult),NA,CT,"00_neuro_brain_angio,na,head,head 00_neuro_bra...",CT,3004287
347,HEAD,NA,Head^00_NEURO_BRAIN_angio (Adult),NA,CT,"na,na,head,head 00_neuro_brain_angio (adult),ct",CT,3004287


In [33]:
finaldata=data[['ProtocolName', 'PerformedProcedureStepDescription', 'BodyPartExamined', 'StudyDescription', 'Modality', 'Protocol_Concept_ID']]

In [34]:
finaldata

,ProtocolName,PerformedProcedureStepDescription,BodyPartExamined,StudyDescription,Modality,Protocol_Concept_ID
0,,NA,,,AP,
4,Brain_without,C-T Brain Routin,HEAD,Head^Brain_without (Adult),CT,3004287
17,NA,NA,UP_GI,NA,RF,
23,NA,RP5101,ABDOMEN,NA,CR,3053099
25,NA,RP3110,CHEST AP,NA,CR,3010713
36,NA,NA,Chest,RP3110,CR,3010713
37,NA,RP5200,KUB,NA,CR,
38,00_NEURO_BRAIN_angio,C-T Brain Angio c Contrast,HEAD,Head^00_NEURO_BRAIN_angio (Adult),CT,3004287
115,00_NEURO_BRAIN_angio,NA,HEAD,Head^00_NEURO_BRAIN_angio (Adult),CT,3004287
347,NA,NA,HEAD,Head^00_NEURO_BRAIN_angio (Adult),CT,3004287


In [35]:
RadiologyOccurrenceTable=pd.merge(left=RadiologyOccurrenceTable, right=finaldata, how='left', on=['ProtocolName', 'PerformedProcedureStepDescription', 'BodyPartExamined', 'StudyDescription', 'Modality'])
del RadiologyOccurrenceTable['ProtocolName']
del RadiologyOccurrenceTable['PerformedProcedureStepDescription']
del RadiologyOccurrenceTable['BodyPartExamined']
del RadiologyOccurrenceTable['StudyDescription']
del RadiologyOccurrenceTable['Modality']

In [36]:
RadiologyOccurrenceTable

,RadiologyOccurrenceID,StudyDateTime,PatientID,Manufacturer,Radiology_Comment,Radiology_Protocol_Concept_ID,imageTotalNo,Protocol_Concept_ID
0,1.2.392.200036.9116.2.6.1.48.1214868379.129460...,2011-01-10 05:48:22,AN_ID_20160905152254,TOSHIBA,,,33,3004287
1,1.2.392.200036.9116.2.6.1.48.1214868379.129460...,2011-01-10 06:20:33,AN_ID_20160905152340,TOSHIBA,,,263,3047709
2,1.2.392.200036.9116.2.6.1.48.1214868379.129510...,2011-01-16 00:05:54,AN_ID_20160905154229,TOSHIBA,,,54,3004287
3,1.2.392.200036.9116.2.6.1.48.1214868379.133568...,2012-04-29 15:57:11,AN_ID_20160921122518,TOSHIBA,,,29,3004287
4,1.2.392.200036.9116.2.6.1.48.1214868379.133568...,2012-04-29 16:00:15,AN_ID_20160921122438,TOSHIBA,,,249,3047709
...,...,...,...,...,...,...,...,...
56,1.3.6.1.4.1.14519.5.2.1.6279.6001.675114969651...,2000-01-01 00:00:00,LIDC-IDRI-0023,GE MEDICAL SYSTEMS,,,2,3010713
57,1.3.6.1.4.1.14519.5.2.1.6279.6001.774731473056...,2000-01-01 00:00:00,LIDC-IDRI-0020,GE MEDICAL SYSTEMS,,,2,3010713
58,1.3.6.1.4.1.14519.5.2.1.6279.6001.960701544671...,2000-01-01 00:00:00,LIDC-IDRI-0010,Philips Medical Systems,,,1,3010713
59,9.1.48564849485154484851,2008-01-07 18:22:12,0710537,DATA HUB,,,2,


In [227]:
# R-CDM made with database from SQL
import pyodbc
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=128.1.99.58;DATABASE=ID_XYLITOL_WHITE;UID=;PWD=')
cursor = cnxn.cursor()
SQL_Query = pd.read_sql_query(
"select * from importantDICOMheader where Modality in ('CR', 'DX') and BodyPartExamined='chest'", cnxn)
chestXRdf = pd.DataFrame(SQL_Query, columns=['Filepath', 'ImageID', 'SeriesID', 'RadiologyOccurrenceID', 'PatientID', 'ImageResolutionRows', 'ImageResolutionColumns', 'SliceThickness', 'ProtocolName', 'PerformedProcedureStepDescription', 'BodyPartExamined', 'Modality', 'SeriesDescription', 'StudyDescription', 'Manufacturer', 'StudyDate', 'StudyTime'])

In [232]:
chestXRdf

,Filepath,ImageID,SeriesID,RadiologyOccurrenceID,PatientID,ImageResolutionRows,ImageResolutionColumns,SliceThickness,ProtocolName,PerformedProcedureStepDescription,BodyPartExamined,Modality,SeriesDescription,StudyDescription,Manufacturer,StudyDate,StudyTime
0,E:\PACS\PACS\bkpnew24\20110604\E7746956\I00242...,4WeTF3VmZQ3BscYaiM443MP8IwLsHh9DnwQUcsp85T2GS5...,kRHGsxTA10eb2TyHtfuCNeEx2gLsijjyeh1oGkGRBoRw60...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby6HB6...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,2460,1950,NA,NA,NA,CHEST,CR,Chest AP,[¿UºICD]CHEST/Rib,Agfa-Gevaert AG,20110604,052703
1,E:\PACS\PACS\bkpnew24\20110605\E7746957\I00242...,4WeTF3VmZQ3BscYaiM443MP8IwLsHh9DnwQUcsp85T2GS5...,kRHGsxTA10eb2TyHtfuCNeEx2gLsijjyeh1oGkGRBoRw60...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby6HB6...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,2460,2970,NA,NA,NA,CHEST,CR,Chest AP,[¿UºICD]CHEST/Rib,Agfa-Gevaert AG,20110605,030004
2,E:\PACS\PACS\bkpnew24\20110605\E7746958\I00242...,4WeTF3VmZQ3BscYaiM443MP8IwLsHh9DnwQUcsp85T2GS5...,kRHGsxTA10eb2TyHtfuCNeEx2gLsijjyeh1oGkGRBoRw60...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby6HB6...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,2460,2970,NA,NA,NA,CHEST,CR,Chest AP,[¿UºICD]CHEST/Rib,Agfa-Gevaert AG,20110605,210635
3,E:\PACS\PACS\bkpnew24\20110606\E7746959\I00242...,4WeTF3VmZQ3BscYaiM443MP8IwLsHh9DnwQUcsp85T2GS5...,kRHGsxTA10eb2TyHtfuCNeEx2gLsijjyeh1oGkGRBoRw60...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby6HB6...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,2460,1950,NA,NA,NA,CHEST,CR,Chest AP,[¿UºICD]CHEST/Rib,Agfa-Gevaert AG,20110606,054106
4,E:\PACS\PACS\bkpnew24\20110607\E7731563\I00242...,4WeTF3VmZQ3BscYaiM443E/2uNrZ6sG2Pe5mpGDa/gFElE...,kRHGsxTA10eb2TyHtfuCNZHY+jWXVLlZaWQ7RqNdrPJJtA...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby7NKv...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,3020,2484,NA,PA,NA,CHEST,DX,PA,[¿UºICD]CHEST PA / LAT (L),Company,20110607,110734.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1812,E:\PACS\PACS\bkpnew24\20081013\E7776103\I00243...,4WeTF3VmZQ3BscYaiM443E/2uNrZ6sG2Pe5mpGDa/gFnf7...,kRHGsxTA10eb2TyHtfuCNZHY+jWXVLlZaWQ7RqNdrPKnuR...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby4uap...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,2140,1760,NA,NA,BOTH LOWER EXTREMITY,CHEST,CR,NA,[¿UºICD]CHEST,"FUJI PHOTO FILM Co., ltd.",20081013,101451.000
1813,E:\PACS\PACS\bkpnew24\20081016\E7782142\I00244...,4WeTF3VmZQ3BscYaiM443MP8IwLsHh9DnwQUcsp85T03Kt...,kRHGsxTA10eb2TyHtfuCNeEx2gLsijjyeh1oGkGRBoQHr3...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby5Ml1...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,2446,2010,NA,Chest AP,NA,CHEST,CR,,[¿UºICD]CHEST AP,KONICA MINOLTA,20081016,103100.0
1814,E:\PACS\PACS\bkpnew24\20081024\E7663524\I00238...,4WeTF3VmZQ3BscYaiM443E/2uNrZ6sG2Pe5mpGDa/gGYV6...,kRHGsxTA10eb2TyHtfuCNZHY+jWXVLlZaWQ7RqNdrPIHdk...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby6gJk...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,2021,1880,NA,CHEST_1_VIEW,Performed Desc,CHEST,CR,Chest,[¿UºICD]CHEST,"""GE Healthcare""",20081024,100422.000
1815,E:\PACS\PACS\bkpnew24\20081029\E7862941\I00247...,4WeTF3VmZQ3BscYaiM443MP8IwLsHh9DnwQUcsp85T15Vx...,kRHGsxTA10eb2TyHtfuCNeEx2gLsijjyeh1oGkGRBoT8Zo...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby4+Nu...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,2446,2010,NA,Chest AP,NA,CHEST,CR,NA,[¿UºICD]Chest AP,KONICA MINOLTA,20081029,174911


In [217]:
SQL_Query = pd.read_sql_query(
"select * from importantDICOMheader where Modality in ('CR', 'DX') and BodyPartExamined='abdomen'", cnxn)
abdXRdf = pd.DataFrame(SQL_Query, columns=['Filepath', 'ImageID', 'SeriesID', 'RadiologyOccurrenceID', 'PatientID', 'ImageResolutionRows', 'ImageResolutionColumns', 'SliceThickness', 'ProtocolName', 'PerformedProcedureStepDescription', 'BodyPartExamined', 'Modality', 'SeriesDescription', 'StudyDescription', 'Manufacturer', 'StudyDate', 'StudyTime'])

In [218]:
abdXRdf

,Filepath,ImageID,SeriesID,RadiologyOccurrenceID,PatientID,ImageResolutionRows,ImageResolutionColumns,SliceThickness,ProtocolName,PerformedProcedureStepDescription,BodyPartExamined,Modality,SeriesDescription,StudyDescription,Manufacturer,StudyDate,StudyTime
0,E:\PACS\PACS\bkpnew24\20111122\E7674331\I00239...,4WeTF3VmZQ3BscYaiM443E/2uNrZ6sG2Pe5mpGDa/gH9Fv...,kRHGsxTA10eb2TyHtfuCNZHY+jWXVLlZaWQ7RqNdrPL/Ya...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby65d8...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,2663,2333,NA,NA,Abdomen E/S/Rt Lat,ABDOMEN,CR,Abdomen Erect,[¿UºICD]Abdomen E/S/Rt Lat,Philips Medical Systems,20111122,103639
1,E:\PACS\PACS\bkpnew24\20111122\E7674331\I00239...,4WeTF3VmZQ3BscYaiM443E/2uNrZ6sG2Pe5mpGDa/gH9Fv...,kRHGsxTA10eb2TyHtfuCNZHY+jWXVLlZaWQ7RqNdrPL/Ya...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby65d8...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,2997,2098,NA,NA,Abdomen E/S/Rt Lat,ABDOMEN,CR,Abdomen lat,[¿UºICD]Abdomen E/S/Rt Lat,Philips Medical Systems,20111122,103639
2,E:\PACS\PACS\bkpnew24\20111122\E7674331\I00239...,4WeTF3VmZQ3BscYaiM443E/2uNrZ6sG2Pe5mpGDa/gH9Fv...,kRHGsxTA10eb2TyHtfuCNZHY+jWXVLlZaWQ7RqNdrPL/Ya...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby65d8...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,3001,2921,NA,NA,Abdomen E/S/Rt Lat,ABDOMEN,CR,Abdomen Supine,[¿UºICD]Abdomen E/S/Rt Lat,Philips Medical Systems,20111122,103639
3,E:\PACS\PACS\bkpnew24\20111123\E7697422\I00240...,4WeTF3VmZQ3BscYaiM443E/2uNrZ6sG2Pe5mpGDa/gEzBk...,kRHGsxTA10eb2TyHtfuCNZHY+jWXVLlZaWQ7RqNdrPIVmD...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby65d8...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,3000,2672,NA,NA,NA,ABDOMEN,DX,ABDOMEN AP (ERECT),[¿UºICD]ABDOMEN SUPINE & ERECT,LISTEM,20111123,112242
4,E:\PACS\PACS\bkpnew24\20111123\E7697422\I00240...,4WeTF3VmZQ3BscYaiM443E/2uNrZ6sG2Pe5mpGDa/gEzBk...,kRHGsxTA10eb2TyHtfuCNZHY+jWXVLlZaWQ7RqNdrPIVmD...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby65d8...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,3000,2736,NA,NA,NA,ABDOMEN,DX,ABDOMEN AP (SUPINE),[¿UºICD]ABDOMEN SUPINE & ERECT,LISTEM,20111123,112242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,E:\PACS\PACS\bkpnew24\20130106\E7814361\I00245...,4WeTF3VmZQ3BscYaiM443E/2uNrZ6sG2Pe5mpGDa/gE9kY...,kRHGsxTA10eb2TyHtfuCNZHY+jWXVLlZaWQ7RqNdrPJr/u...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby5sBY...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,2976,2396,NA,NA,NA,ABDOMEN,CR,NA,[¿UºICD]ABDOMEN Erect & Supine,COMED Medical Systems,20130106,125827.0
522,E:\PACS\PACS\bkpnew24\20130103\E7709987\I00241...,aYUq+8r2im7+VByFzpOCifg6YjXUwoFy2XqLEONR6yAEU3...,mnb0oeZhKf05ZN7xrSQHZD3oMxjEfj55RcclJgcU23rJEV...,90wypvsLSe6mjyeWrrRu+6gw41WDXpkDOLr+KlI5twTKgK...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,1760,1760,NA,NA,ABDOMEN SUPINE,ABDOMEN,CR,NA,NA,FUJIFILM Corporation,20130103,113132.000
523,E:\PACS\PACS\bkpnew24\20090901\E7840631\I00246...,4WeTF3VmZQ3BscYaiM443E/2uNrZ6sG2Pe5mpGDa/gHIW6...,kRHGsxTA10eb2TyHtfuCNZHY+jWXVLlZaWQ7RqNdrPIrG+...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby62dy...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,2140,1760,NA,NA,Abdomen Supine & Erect,ABDOMEN,CR,NA,[¿UºICD]Abdomen,FUJIFILM Corporation,20090901,120712.000
524,E:\PACS\PACS\bkpnew24\20090901\E7840631\I00246...,4WeTF3VmZQ3BscYaiM443E/2uNrZ6sG2Pe5mpGDa/gHIW6...,kRHGsxTA10eb2TyHtfuCNZHY+jWXVLlZaWQ7RqNdrPIrG+...,GnPNqCtLxLJ4DZZc+R6UklR1grLBwcaKrzggzmSzby62dy...,90wypvsLSe6mjyeWrrRu+/dMMqb7C0nupo8nlq60bvv3TD...,2140,1760,NA,NA,Abdomen Supine & Erect,ABDOMEN,CR,NA,[¿UºICD]Abdomen,FUJIFILM Corporation,20090901,120712.000


In [229]:
SQL_Query = pd.read_sql_query(
"select * from importantDICOMheader where Modality in ('CT') and BodyPartExamined='head'", cnxn)
headCTdf = pd.DataFrame(SQL_Query, columns=['Filepath', 'ImageID', 'SeriesID', 'RadiologyOccurrenceID', 'PatientID', 'ImageResolutionRows', 'ImageResolutionColumns', 'SliceThickness', 'ProtocolName', 'PerformedProcedureStepDescription', 'BodyPartExamined', 'Modality', 'SeriesDescription', 'StudyDescription', 'Manufacturer', 'StudyDate', 'StudyTime'])